In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
import pandas as pd
import numpy as np

# own Modules 
from anomaly_classifier import AnomalyClassifier

# MLE Mode: 

In [2]:
mode = "MLE"
threshold = "2sigma"

# MSE Mode:

In [91]:
mode = "MSE"
threshold = "mean_plus_{}".format("60percent")

## Parameters phm data

In [92]:
param = {
    "data" : {
        "path" : '../../data/phm_data_challenge/recipe/dataset_for_each_recipe/test/test_recipe_67.csv',
        "droped_feature" : ["stage", "Lot", "runnum", "recipe", "recipe_step",
                            "up time", "ongoing time", 
                            "ETCHSOURCEUSAGE", "ETCHAUXSOURCETIMER", 
                            "ETCHAUX2SOURCETIMER", "FIXTURESHUTTERPOSITION", "ROTATIONSPEED"
                           ],
        "features_not_to_scale": []
    },
    "model" : {
        "input_size" : 12,
    },
        "anomaly_detection" : {
        "threshold_quadratic_difference" : 0.5,
        "threshold_q_d_each_sensor" : [0.1010492374011701, 0.11756227450933948, 0.07937848364476605, 0.10853243510317657, 
                                       0.10570055771211723, 0.0857727231192026, 0.07568253576225717, 0.09385799791533005, 
                                       0.13053110110886396, 0.07788286969806985, 0.07917948961852328, 0.06592829119438347],
        "no_standard_deviation" : 2
    },
    "prediction_path": "../visualisation/files/prediction/{}/phm_67.csv".format(mode),
    "classification_path" : "../visualisation/files/classification/{}/phm67_{}.csv".format(mode, threshold)
}

## Parameters artifical data

In [75]:
param = {
    "data" : {
        "path" : '../../data/artifical_signals/artifical_2_signals_errors.csv',
        "droped_feature" : ["anomaly"
                           ],
        "features_not_to_scale": []
    },
    "model" : {
        "input_size" : 2,
    },
        "anomaly_detection" : {
        "threshold_quadratic_difference" : 0.5,
        "threshold_q_d_each_sensor" : [0.13679444326055645, 0.5441877437706729],
        "no_standard_deviation" : 2
    },
    "prediction_path": "../visualisation/files/prediction/{}/artfic.csv".format(mode),
    "classification_path" : "../visualisation/files/classification/{}/artfic_{}.csv".format(mode, threshold)
}

## Parameters cpps data

In [70]:
param = {
    "data" : {
        "path" : '../../data/cpps_degradation_new/data_obs10/test/obs_space_error_sinusiod_preprocessed.csv',
        "droped_feature" : ["status"
                           ],
        "features_not_to_scale": []
    },
    "model" : {
        "input_size" : 10,
    },
        "anomaly_detection" : {
        "threshold_quadratic_difference" : 0.5,
        "threshold_q_d_each_sensor" : [0.24372421389777016, 0.24761591192911026, 0.32903928384871034, 
                                       0.18159441767379658, 0.27133063146032477, 0.23155811781422755, 
                                       0.149805263381253, 0.31187742832928145, 0.0716386888894709, 0.2080102174959789],
        "no_standard_deviation" : 2
    },
    "prediction_path": "../visualisation/files/prediction/{}/cpps.csv".format(mode),
    "classification_path" : "../visualisation/files/classification/{}/cpps_{}.csv".format(mode, threshold)
}

## Detect Anomaly
### Euclidean Distance Metric for the Hole Machine (for MSE Loss Function)

In [34]:
results_prediction = pd.read_csv(param["prediction_path"], sep=";")
anomaly_classifier = AnomalyClassifier()
results = anomaly_classifier.apply_euclidean_distance_metric(results_prediction, 
                                                           param['model']['input_size'], 
                                                           param["anomaly_detection"]["threshold_quadratic_difference"])

### Euclidean Distance Difference Metric for Each Sensor (for MSE Loss Function)

In [93]:
results_prediction = pd.read_csv(param["prediction_path"], sep=";")
anomaly_classifier = AnomalyClassifier()
results = anomaly_classifier.apply_euclidean_distance_metric(results_prediction, 
                                                           param['model']['input_size'],
                                                           param["anomaly_detection"]["threshold_quadratic_difference"],
                                                           percentage = 1.6,
                                                           each_sensor=True, 
                                                           threshold_sensors=param["anomaly_detection"]["threshold_q_d_each_sensor"])

### Standard Deviation Metric (for MLE Loss Function)

In [ ]:
results_prediction = pd.read_csv(param["prediction_path"], sep=";")
anomaly_classifier = AnomalyClassifier()
results = anomaly_classifier.apply_prediction_interval_metric(results_prediction, 
                                                            param['model']['input_size'],
                                                            param["anomaly_detection"]["no_standard_deviation"])

In [37]:
results.head()

,ID,sine_signal target,sawtooth_signal target,sine_signal predicted,sawtooth_signal predicted,sine_signal reconstruction error,sawtooth_signal reconstruction error,Anomaly Sensor_1,Anomaly Sensor_2
0,8.0,1.253286,-0.986644,1.237256,-0.277459,0.016029,0.709186,0,1
1,9.0,1.313607,-0.618104,1.304106,-0.529158,0.009501,0.088946,0,0
2,10.0,1.429184,-0.357066,1.316485,-0.459979,0.112699,0.102914,0,0
3,11.0,1.376550,0.055021,1.388720,-0.267551,0.012169,0.322571,0,0
4,12.0,1.164972,0.153692,1.383462,0.145293,0.218489,0.008399,1,0


## Combine prediction data with data which was not consider for inference

In [94]:
original_sensor_data = pd.read_csv(param["data"]["path"])
data_of_droped_feature = original_sensor_data.loc[:, param["data"]["droped_feature"]+["ID"]]
complete_data = results.merge(right=data_of_droped_feature, how="inner", on="ID")

## Save data to csv file

In [ ]:
complete_data.to_csv(param["classification_path"], sep=";", index=False)